In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import RobustScaler,PowerTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'      

scalers = {}
# Function to center, scale, and return a series
def scale_group(group):
    scaler = RobustScaler(with_centering=False)
    #scaler = PowerTransformer()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)


In [ ]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
#import dask.dataframe as dd
df_final = pd.read_parquet(DATOS_DIR+'FE_10_dataset.parquet') 
#df_final = df_final.compute()

df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)




In [ ]:

## ESTO ES UN HACK PORQUE ESTA MAL ARMADA LA FUNCIÓN DE ESCALADO Y SOLO GUARDA LOS DATOS DE LA ÜLTIMA APLICACIÓN

with open(DATOS_DIR + '/scalers.pkl', 'rb') as file:
    scalers = pickle.load(file)

#df_test = df_final[(df_final['product_id'] == 21276) & (df_final['customer_id'] == 10029)][['product_id','tn', 'weight']]
#df_test['tn'] = df_test.groupby('product_id')['tn'].transform(inverse_scale_group)

#df_test

In [ ]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-11-01']


In [ ]:
df_true = df_true[['product_id','customer_id','tn']]
df_true

In [ ]:
#df_true['tn'] = df_true.groupby(['product_id','customer_id'])['tn'].transform(inverse_scale_group)
#pred_true = df_true.groupby(['product_id'])['tn'].sum()
#pred_true.to_csv(DATOS_DIR+'/pred/TRUE-NOVEMBER.csv', index=True,header=True)


In [ ]:
#df_final[df_final['product_id'] == 20402 & df_final['customer_id'] == 10479][['product_id','tn']]

In [ ]:

#Filtro test
#
#
# df_final = df_final[df_final['product_id'] < 20013]
#df_final = df_final[df_final['product_id'] == 20072]

weight= df_final[['weight','product_id','customer_id']]
#df_final.drop(columns=['weight'], inplace=True)
#filtered_weight = weight[weight['weight'] != 0]
#weight_mean = filtered_weight.groupby('product_id')['weight'].mean().to_dict()
#weight['weight'] = weight['product_id'].map(weight_mean)

weight = weight.groupby(['product_id','customer_id'])['weight'].mean()
weight = weight.reset_index()
df_final = df_final.drop(columns=['weight'])

In [ ]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        'seed': 113,
        #'learning_rate': 0.2,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        'min_data_per_leaf': 25,
        'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

df_final['tn_2'].fillna(0, inplace=True)
predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0

for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    
    product_weights = pd.merge(df_producto.reset_index(), weight, on=['product_id', 'customer_id'], how='left')
    product_weights.set_index('periodo', inplace=True)
    weight_p = product_weights['weight']
    
    df_producto['tn_2'] = df_producto['tn_2'].fillna(0)
  
    #############################################################################  
    #### Agrupar y escalar
   
    model, average_metric = labo.train_lightgbm_model(df_producto,params,metric='rmse',
                                                      weights=weight_p)
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month_customer(model, last_data_points)
    preds = predictions.groupby(['product_id'])['tn_2'].transform(inverse_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


In [ ]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0028-prediccion-RSME_scaled_Robust-product_id_LSTM-Back_In_Black.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)
